In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("datasets/", one_hot=True)

Extracting datasets/train-images-idx3-ubyte.gz
Extracting datasets/train-labels-idx1-ubyte.gz
Extracting datasets/t10k-images-idx3-ubyte.gz
Extracting datasets/t10k-labels-idx1-ubyte.gz


In [3]:
# input variable
X = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32, name="X")
X

<tf.Tensor 'X:0' shape=(?, 28, 28, 1) dtype=float32>

In [4]:
y = tf.placeholder(shape=[None, 10], dtype=tf.float32, name="y")
y

<tf.Tensor 'y:0' shape=(?, 10) dtype=float32>

In [5]:
prob = tf.placeholder(dtype=tf.float32, name="prob")

In [6]:
# first convolutional layer
conv1 = tf.layers.conv2d(X, filters=64, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu, name="conv1")
conv1

<tf.Tensor 'conv1/Relu:0' shape=(?, 28, 28, 64) dtype=float32>

In [7]:
# first pooling layer
pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], padding="same", strides=2, name="pool1")
pool1

<tf.Tensor 'pool1/MaxPool:0' shape=(?, 14, 14, 64) dtype=float32>

In [8]:
# second convolutional layer
conv2 = tf.layers.conv2d(pool1, filters=128, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu, name="conv2")
conv2

<tf.Tensor 'conv2/Relu:0' shape=(?, 14, 14, 128) dtype=float32>

In [9]:
# second pooling layer
pool2 = tf.layers.max_pooling2d(conv2, pool_size=[2, 2], strides=2, padding="same", name="pool2")
pool2

<tf.Tensor 'pool2/MaxPool:0' shape=(?, 7, 7, 128) dtype=float32>

In [10]:
# third convolutional layer
conv3 = tf.layers.conv2d(pool2, filters=256, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu, name="conv3")
conv3

<tf.Tensor 'conv3/Relu:0' shape=(?, 7, 7, 256) dtype=float32>

In [11]:
# third pooling layer
pool3 = tf.layers.max_pooling2d(conv3, pool_size=[2, 2], strides=2, padding="same", name="pool3")
pool3

<tf.Tensor 'pool3/MaxPool:0' shape=(?, 4, 4, 256) dtype=float32>

In [12]:
# Dense layer
pool2_flat = tf.reshape(pool3, [-1, 4 * 4 * 256], name="pool2_flat")
pool2_flat

<tf.Tensor 'pool2_flat:0' shape=(?, 4096) dtype=float32>

In [13]:
dense = tf.layers.dense(pool2_flat, units=1024, activation=tf.nn.relu, name="dense")
dense

<tf.Tensor 'dense/Relu:0' shape=(?, 1024) dtype=float32>

In [14]:
dropout = tf.layers.dropout(dense, prob, name="dropout")
dropout

<tf.Tensor 'dropout/Identity:0' shape=(?, 1024) dtype=float32>

In [15]:
logits = tf.layers.dense(dropout, 10)
logits

<tf.Tensor 'dense_2/BiasAdd:0' shape=(?, 10) dtype=float32>

In [16]:
y_pred = tf.nn.softmax(logits, name="y_pred")
y_pred

<tf.Tensor 'y_pred:0' shape=(?, 10) dtype=float32>

In [17]:
# Loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits))
loss

<tf.Tensor 'Mean:0' shape=() dtype=float32>

In [18]:
# Training operation
training_op = tf.train.AdamOptimizer(0.001).minimize(loss)

In [19]:
# Accuracy
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [20]:
# Initialize and save
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [23]:
# Training
n_epochs = 1
batch_size = 32
restore_checkpoint = True

n_iterations_per_epoch = mnist.train.num_examples // batch_size
n_iterations_validation = mnist.validation.num_examples // batch_size
best_loss_val = np.infty
checkpoint_path = "my_cnn/cnn.ckpt"
summary_path = "my_cnn/"

with tf.Session() as sess:    
    if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
        saver.restore(sess, checkpoint_path)
    else:
        init.run()
        for epoch in range(n_epochs):
            for iteration in range(1, n_iterations_per_epoch + 1):
                X_batch, y_batch = mnist.train.next_batch(batch_size)


                # Run the training operation and measure the loss:
                _, loss_train, acc = sess.run([training_op, loss, accuracy],
                                          feed_dict={X: X_batch.reshape([-1, 28, 28, 1]),
                                                     y: y_batch,
                                                     prob: 0.5 })

                print("\rIteration: {}/{} ({:.1f}%)  Accuracy:{:.2f}%  Loss: {:.6f}".format(iteration,
                                                                          n_iterations_per_epoch,
                                                                          iteration * 100 / n_iterations_per_epoch,
                                                                          acc * 100,
                                                                          loss_train))
            # At the end of each epoch,
            # measure the validation loss and accuracy:
            loss_vals = []
            acc_vals = []
            for iteration in range(1, n_iterations_validation + 1):
                X_batch, y_batch = mnist.validation.next_batch(batch_size)
                loss_val, acc_val = sess.run([loss, accuracy],
                                             feed_dict={X: X_batch.reshape([-1, 28, 28, 1]),
                                                        y: y_batch,
                                                        prob: 1.0})
                loss_vals.append(loss_val)
                acc_vals.append(acc_val)
                print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                          iteration, n_iterations_validation,
                          iteration * 100 / n_iterations_validation),
                      end=" " * 10)
            loss_val = np.mean(loss_vals)
            acc_val = np.mean(acc_vals)
            print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f}{}".format(
                        epoch + 1, acc_val * 100, loss_val, " (improved)" if loss_val < best_loss_val else ""))

            # And save the model if it improved:
            if loss_val < best_loss_val:
                save_path = saver.save(sess, checkpoint_path)
                best_loss_val = loss_val

                writer = tf.summary.FileWriter(summary_path, sess.graph)

Iteration: 1/1718 (0.1%)  Accuracy:15.62%  Loss: 2.302060
Iteration: 2/1718 (0.1%)  Accuracy:6.25%  Loss: 2.453395
Iteration: 3/1718 (0.2%)  Accuracy:37.50%  Loss: 2.250383
Iteration: 4/1718 (0.2%)  Accuracy:12.50%  Loss: 2.213284
Iteration: 5/1718 (0.3%)  Accuracy:9.38%  Loss: 2.252246
Iteration: 6/1718 (0.3%)  Accuracy:21.88%  Loss: 2.253464
Iteration: 7/1718 (0.4%)  Accuracy:21.88%  Loss: 2.244262
Iteration: 8/1718 (0.5%)  Accuracy:37.50%  Loss: 2.127113
Iteration: 9/1718 (0.5%)  Accuracy:53.12%  Loss: 2.048895
Iteration: 10/1718 (0.6%)  Accuracy:46.88%  Loss: 1.881180
Iteration: 11/1718 (0.6%)  Accuracy:50.00%  Loss: 1.792043
Iteration: 12/1718 (0.7%)  Accuracy:50.00%  Loss: 1.634714
Iteration: 13/1718 (0.8%)  Accuracy:53.12%  Loss: 1.634261
Iteration: 14/1718 (0.8%)  Accuracy:34.38%  Loss: 1.472763
Iteration: 15/1718 (0.9%)  Accuracy:62.50%  Loss: 1.130898
Iteration: 16/1718 (0.9%)  Accuracy:81.25%  Loss: 0.782785
Iteration: 17/1718 (1.0%)  Accuracy:65.62%  Loss: 1.095692
Iteratio

Iteration: 140/1718 (8.1%)  Accuracy:90.62%  Loss: 0.375120
Iteration: 141/1718 (8.2%)  Accuracy:96.88%  Loss: 0.093316
Iteration: 142/1718 (8.3%)  Accuracy:96.88%  Loss: 0.090609
Iteration: 143/1718 (8.3%)  Accuracy:100.00%  Loss: 0.019166
Iteration: 144/1718 (8.4%)  Accuracy:100.00%  Loss: 0.016287
Iteration: 145/1718 (8.4%)  Accuracy:93.75%  Loss: 0.185084
Iteration: 146/1718 (8.5%)  Accuracy:96.88%  Loss: 0.164537
Iteration: 147/1718 (8.6%)  Accuracy:96.88%  Loss: 0.075493
Iteration: 148/1718 (8.6%)  Accuracy:93.75%  Loss: 0.159070
Iteration: 149/1718 (8.7%)  Accuracy:96.88%  Loss: 0.133236
Iteration: 150/1718 (8.7%)  Accuracy:96.88%  Loss: 0.107008
Iteration: 151/1718 (8.8%)  Accuracy:100.00%  Loss: 0.042334
Iteration: 152/1718 (8.8%)  Accuracy:96.88%  Loss: 0.167820
Iteration: 153/1718 (8.9%)  Accuracy:90.62%  Loss: 0.226575
Iteration: 154/1718 (9.0%)  Accuracy:100.00%  Loss: 0.047141
Iteration: 155/1718 (9.0%)  Accuracy:93.75%  Loss: 0.150210
Iteration: 156/1718 (9.1%)  Accuracy

Iteration: 275/1718 (16.0%)  Accuracy:96.88%  Loss: 0.182282
Iteration: 276/1718 (16.1%)  Accuracy:96.88%  Loss: 0.159097
Iteration: 277/1718 (16.1%)  Accuracy:87.50%  Loss: 0.142917
Iteration: 278/1718 (16.2%)  Accuracy:96.88%  Loss: 0.131359
Iteration: 279/1718 (16.2%)  Accuracy:93.75%  Loss: 0.196086
Iteration: 280/1718 (16.3%)  Accuracy:93.75%  Loss: 0.285650
Iteration: 281/1718 (16.4%)  Accuracy:100.00%  Loss: 0.023236
Iteration: 282/1718 (16.4%)  Accuracy:96.88%  Loss: 0.048967
Iteration: 283/1718 (16.5%)  Accuracy:87.50%  Loss: 0.438673
Iteration: 284/1718 (16.5%)  Accuracy:100.00%  Loss: 0.017204
Iteration: 285/1718 (16.6%)  Accuracy:93.75%  Loss: 0.195146
Iteration: 286/1718 (16.6%)  Accuracy:96.88%  Loss: 0.058649
Iteration: 287/1718 (16.7%)  Accuracy:93.75%  Loss: 0.077348
Iteration: 288/1718 (16.8%)  Accuracy:96.88%  Loss: 0.128147
Iteration: 289/1718 (16.8%)  Accuracy:93.75%  Loss: 0.197669
Iteration: 290/1718 (16.9%)  Accuracy:93.75%  Loss: 0.092982
Iteration: 291/1718 (1

Iteration: 409/1718 (23.8%)  Accuracy:96.88%  Loss: 0.114539
Iteration: 410/1718 (23.9%)  Accuracy:96.88%  Loss: 0.054682
Iteration: 411/1718 (23.9%)  Accuracy:100.00%  Loss: 0.044002
Iteration: 412/1718 (24.0%)  Accuracy:90.62%  Loss: 0.220882
Iteration: 413/1718 (24.0%)  Accuracy:100.00%  Loss: 0.020920
Iteration: 414/1718 (24.1%)  Accuracy:100.00%  Loss: 0.016284
Iteration: 415/1718 (24.2%)  Accuracy:96.88%  Loss: 0.075798
Iteration: 416/1718 (24.2%)  Accuracy:93.75%  Loss: 0.077084
Iteration: 417/1718 (24.3%)  Accuracy:96.88%  Loss: 0.035426
Iteration: 418/1718 (24.3%)  Accuracy:100.00%  Loss: 0.018297
Iteration: 419/1718 (24.4%)  Accuracy:96.88%  Loss: 0.048786
Iteration: 420/1718 (24.4%)  Accuracy:96.88%  Loss: 0.038535
Iteration: 421/1718 (24.5%)  Accuracy:90.62%  Loss: 0.140637
Iteration: 422/1718 (24.6%)  Accuracy:100.00%  Loss: 0.016586
Iteration: 423/1718 (24.6%)  Accuracy:96.88%  Loss: 0.075049
Iteration: 424/1718 (24.7%)  Accuracy:100.00%  Loss: 0.006946
Iteration: 425/171

Iteration: 543/1718 (31.6%)  Accuracy:93.75%  Loss: 0.304460
Iteration: 544/1718 (31.7%)  Accuracy:100.00%  Loss: 0.022066
Iteration: 545/1718 (31.7%)  Accuracy:100.00%  Loss: 0.024920
Iteration: 546/1718 (31.8%)  Accuracy:96.88%  Loss: 0.064643
Iteration: 547/1718 (31.8%)  Accuracy:96.88%  Loss: 0.057440
Iteration: 548/1718 (31.9%)  Accuracy:93.75%  Loss: 0.081325
Iteration: 549/1718 (32.0%)  Accuracy:100.00%  Loss: 0.024650
Iteration: 550/1718 (32.0%)  Accuracy:96.88%  Loss: 0.156788
Iteration: 551/1718 (32.1%)  Accuracy:100.00%  Loss: 0.038109
Iteration: 552/1718 (32.1%)  Accuracy:96.88%  Loss: 0.075125
Iteration: 553/1718 (32.2%)  Accuracy:93.75%  Loss: 0.104867
Iteration: 554/1718 (32.2%)  Accuracy:100.00%  Loss: 0.032125
Iteration: 555/1718 (32.3%)  Accuracy:96.88%  Loss: 0.074025
Iteration: 556/1718 (32.4%)  Accuracy:96.88%  Loss: 0.075813
Iteration: 557/1718 (32.4%)  Accuracy:93.75%  Loss: 0.163509
Iteration: 558/1718 (32.5%)  Accuracy:90.62%  Loss: 0.120830
Iteration: 559/1718

...

Iteration: 1077/1718 (62.7%)  Accuracy:96.88%  Loss: 0.141513
Iteration: 1078/1718 (62.7%)  Accuracy:100.00%  Loss: 0.020417
Iteration: 1079/1718 (62.8%)  Accuracy:96.88%  Loss: 0.042279
Iteration: 1080/1718 (62.9%)  Accuracy:96.88%  Loss: 0.136795
Iteration: 1081/1718 (62.9%)  Accuracy:100.00%  Loss: 0.053097
Iteration: 1082/1718 (63.0%)  Accuracy:93.75%  Loss: 0.283253
Iteration: 1083/1718 (63.0%)  Accuracy:93.75%  Loss: 0.187036
Iteration: 1084/1718 (63.1%)  Accuracy:96.88%  Loss: 0.076057
Iteration: 1085/1718 (63.2%)  Accuracy:96.88%  Loss: 0.139121
Iteration: 1086/1718 (63.2%)  Accuracy:100.00%  Loss: 0.018224
Iteration: 1087/1718 (63.3%)  Accuracy:87.50%  Loss: 0.269435
Iteration: 1088/1718 (63.3%)  Accuracy:100.00%  Loss: 0.007833
Iteration: 1089/1718 (63.4%)  Accuracy:100.00%  Loss: 0.008361
Iteration: 1090/1718 (63.4%)  Accuracy:96.88%  Loss: 0.075066
Iteration: 1091/1718 (63.5%)  Accuracy:96.88%  Loss: 0.120282
Iteration: 1092/1718 (63.6%)  Accuracy:100.00%  Loss: 0.023516
It

Iteration: 1208/1718 (70.3%)  Accuracy:100.00%  Loss: 0.027131
Iteration: 1209/1718 (70.4%)  Accuracy:93.75%  Loss: 0.239752
Iteration: 1210/1718 (70.4%)  Accuracy:100.00%  Loss: 0.029692
Iteration: 1211/1718 (70.5%)  Accuracy:93.75%  Loss: 0.160925
Iteration: 1212/1718 (70.5%)  Accuracy:100.00%  Loss: 0.004079
Iteration: 1213/1718 (70.6%)  Accuracy:100.00%  Loss: 0.006671
Iteration: 1214/1718 (70.7%)  Accuracy:96.88%  Loss: 0.119867
Iteration: 1215/1718 (70.7%)  Accuracy:93.75%  Loss: 0.071981
Iteration: 1216/1718 (70.8%)  Accuracy:96.88%  Loss: 0.086375
Iteration: 1217/1718 (70.8%)  Accuracy:100.00%  Loss: 0.034959
Iteration: 1218/1718 (70.9%)  Accuracy:100.00%  Loss: 0.086549
Iteration: 1219/1718 (71.0%)  Accuracy:100.00%  Loss: 0.042380
Iteration: 1220/1718 (71.0%)  Accuracy:96.88%  Loss: 0.056778
Iteration: 1221/1718 (71.1%)  Accuracy:100.00%  Loss: 0.018513
Iteration: 1222/1718 (71.1%)  Accuracy:100.00%  Loss: 0.021195
Iteration: 1223/1718 (71.2%)  Accuracy:100.00%  Loss: 0.01385

Iteration: 1339/1718 (77.9%)  Accuracy:100.00%  Loss: 0.012090
Iteration: 1340/1718 (78.0%)  Accuracy:100.00%  Loss: 0.013175
Iteration: 1341/1718 (78.1%)  Accuracy:96.88%  Loss: 0.075952
Iteration: 1342/1718 (78.1%)  Accuracy:100.00%  Loss: 0.005803
Iteration: 1343/1718 (78.2%)  Accuracy:100.00%  Loss: 0.015457
Iteration: 1344/1718 (78.2%)  Accuracy:93.75%  Loss: 0.151569
Iteration: 1345/1718 (78.3%)  Accuracy:100.00%  Loss: 0.000986
Iteration: 1346/1718 (78.3%)  Accuracy:96.88%  Loss: 0.176874
Iteration: 1347/1718 (78.4%)  Accuracy:96.88%  Loss: 0.050202
Iteration: 1348/1718 (78.5%)  Accuracy:96.88%  Loss: 0.245984
Iteration: 1349/1718 (78.5%)  Accuracy:100.00%  Loss: 0.026588
Iteration: 1350/1718 (78.6%)  Accuracy:93.75%  Loss: 0.083899
Iteration: 1351/1718 (78.6%)  Accuracy:100.00%  Loss: 0.000946
Iteration: 1352/1718 (78.7%)  Accuracy:100.00%  Loss: 0.011431
Iteration: 1353/1718 (78.8%)  Accuracy:96.88%  Loss: 0.055491
Iteration: 1354/1718 (78.8%)  Accuracy:100.00%  Loss: 0.027408

Iteration: 1470/1718 (85.6%)  Accuracy:96.88%  Loss: 0.053017
Iteration: 1471/1718 (85.6%)  Accuracy:96.88%  Loss: 0.047280
Iteration: 1472/1718 (85.7%)  Accuracy:96.88%  Loss: 0.150891
Iteration: 1473/1718 (85.7%)  Accuracy:100.00%  Loss: 0.028115
Iteration: 1474/1718 (85.8%)  Accuracy:100.00%  Loss: 0.011072
Iteration: 1475/1718 (85.9%)  Accuracy:100.00%  Loss: 0.001255
Iteration: 1476/1718 (85.9%)  Accuracy:100.00%  Loss: 0.004055
Iteration: 1477/1718 (86.0%)  Accuracy:100.00%  Loss: 0.010981
Iteration: 1478/1718 (86.0%)  Accuracy:100.00%  Loss: 0.024985
Iteration: 1479/1718 (86.1%)  Accuracy:100.00%  Loss: 0.005525
Iteration: 1480/1718 (86.1%)  Accuracy:100.00%  Loss: 0.000835
Iteration: 1481/1718 (86.2%)  Accuracy:96.88%  Loss: 0.026561
Iteration: 1482/1718 (86.3%)  Accuracy:100.00%  Loss: 0.002956
Iteration: 1483/1718 (86.3%)  Accuracy:96.88%  Loss: 0.039063
Iteration: 1484/1718 (86.4%)  Accuracy:100.00%  Loss: 0.002515
Iteration: 1485/1718 (86.4%)  Accuracy:96.88%  Loss: 0.16184

Iteration: 1601/1718 (93.2%)  Accuracy:96.88%  Loss: 0.113470
Iteration: 1602/1718 (93.2%)  Accuracy:100.00%  Loss: 0.006834
Iteration: 1603/1718 (93.3%)  Accuracy:100.00%  Loss: 0.017752
Iteration: 1604/1718 (93.4%)  Accuracy:100.00%  Loss: 0.022639
Iteration: 1605/1718 (93.4%)  Accuracy:96.88%  Loss: 0.148666
Iteration: 1606/1718 (93.5%)  Accuracy:96.88%  Loss: 0.095933
Iteration: 1607/1718 (93.5%)  Accuracy:100.00%  Loss: 0.025290
Iteration: 1608/1718 (93.6%)  Accuracy:96.88%  Loss: 0.083364
Iteration: 1609/1718 (93.7%)  Accuracy:96.88%  Loss: 0.132223
Iteration: 1610/1718 (93.7%)  Accuracy:100.00%  Loss: 0.028685
Iteration: 1611/1718 (93.8%)  Accuracy:100.00%  Loss: 0.011765
Iteration: 1612/1718 (93.8%)  Accuracy:96.88%  Loss: 0.147179
Iteration: 1613/1718 (93.9%)  Accuracy:100.00%  Loss: 0.003174
Iteration: 1614/1718 (93.9%)  Accuracy:100.00%  Loss: 0.016208
Iteration: 1615/1718 (94.0%)  Accuracy:96.88%  Loss: 0.062963
Iteration: 1616/1718 (94.1%)  Accuracy:100.00%  Loss: 0.007928

In [25]:
# Evaluation
n_iterations_test = mnist.test.num_examples // batch_size

with tf.Session() as sess:
    
    saver.restore(sess, checkpoint_path)
    
    loss_tests = []
    acc_tests = []
    
    for iteration in range(n_iterations_test + 1):
        X_batch, y_batch = mnist.test.next_batch(batch_size)
        
        loss_test, acc_test = sess.run([loss, accuracy],
                                       feed_dict = {X: X_batch.reshape([-1, 28, 28, 1]),
                                                    y: y_batch,
                                                    prob: 1.0})
        loss_tests.append(loss_test)
        acc_tests.append(acc_test)
        
        
        print("\nEvaluating the model: {}/{} ({:.1f}%)".format(
                    iteration, n_iterations_test, iteration * 100 / n_iterations_test), end=" " * 10)
        
    loss_test = np.mean(loss_tests)
    acc_test = np.mean(acc_tests)
        
    print("\rFinal test accuracy: {: 4f}%  Loss: {:.6f}".format(acc_test * 100, loss_test))

INFO:tensorflow:Restoring parameters from my_cnn/cnn.ckpt

Evaluating the model: 0/312 (0.0%)          
Evaluating the model: 1/312 (0.3%)          
Evaluating the model: 2/312 (0.6%)          
Evaluating the model: 3/312 (1.0%)          
Evaluating the model: 4/312 (1.3%)          
Evaluating the model: 5/312 (1.6%)          
Evaluating the model: 6/312 (1.9%)          
Evaluating the model: 7/312 (2.2%)          
Evaluating the model: 8/312 (2.6%)          
Evaluating the model: 9/312 (2.9%)          
Evaluating the model: 10/312 (3.2%)          
Evaluating the model: 11/312 (3.5%)          
Evaluating the model: 12/312 (3.8%)          
Evaluating the model: 13/312 (4.2%)          
Evaluating the model: 14/312 (4.5%)          
Evaluating the model: 15/312 (4.8%)          
Evaluating the model: 16/312 (5.1%)          
Evaluating the model: 17/312 (5.4%)          
Evaluating the model: 18/312 (5.8%)          
Evaluating the model: 19/312 (6.1%)          
Evaluating the model: 20/312 (6

Evaluating the model: 172/312 (55.1%)          
Evaluating the model: 173/312 (55.4%)          
Evaluating the model: 174/312 (55.8%)          
Evaluating the model: 175/312 (56.1%)          
Evaluating the model: 176/312 (56.4%)          
Evaluating the model: 177/312 (56.7%)          
Evaluating the model: 178/312 (57.1%)          
Evaluating the model: 179/312 (57.4%)          
Evaluating the model: 180/312 (57.7%)          
Evaluating the model: 181/312 (58.0%)          
Evaluating the model: 182/312 (58.3%)          
Evaluating the model: 183/312 (58.7%)          
Evaluating the model: 184/312 (59.0%)          
Evaluating the model: 185/312 (59.3%)          
Evaluating the model: 186/312 (59.6%)          
Evaluating the model: 187/312 (59.9%)          
Evaluating the model: 188/312 (60.3%)          
Evaluating the model: 189/312 (60.6%)          
Evaluating the model: 190/312 (60.9%)          
Evaluating the model: 191/312 (61.2%)          
Evaluating the model: 192/312 (61.5%)   